In [2]:
library(tidyverse)
library(sva)

source("../../utils/plots_eda.R")

# Load data

In [3]:
datasets = c("lab_A", "lab_B", "lab_C",  "lab_D", "lab_E")

In [11]:
all_metadata <- read.table("before/all_design.tsv", header = TRUE, sep = "\t")
all_expression <- read.table("before/all_expr_for_correction.tsv", header = TRUE, sep = "\t") %>% column_to_rownames("sample_id") %>% t() %>% as.data.frame()

all_expression <- all_expression[, all_metadata$sample_id]

# remove NAs rows
all_expression <- na.omit(all_expression)
# remove genes with 0 variance
all_expression <- all_expression[apply(all_expression, 1, var) > 0, ]

In [12]:
print("Loading metadata and expression data")
print("Metadata dimensions:")
print(dim(all_metadata))
print("Expression data dimensions:")
print(dim(all_expression))

[1] "Loading metadata and expression data"
[1] "Metadata dimensions:"
[1] 98  3
[1] "Expression data dimensions:"
[1] 2067   98


In [13]:
design <- model.matrix(~all_metadata$Status)

corrected_expr <- sva::ComBat(dat = all_expression, 
                              batch = all_metadata$batch, 
                              mod = design)

corrected_expr <- as.data.frame(corrected_expr)


Found5batches

Adjusting for1covariate(s) or covariate level(s)

Standardizing Data across genes

Fitting L/S model and finding priors

Finding parametric adjustments

Adjusting the Data




In [14]:
all_metadata$Status <- all_metadata$Status
all_metadata$Dataset <- as.character(all_metadata$batch)

In [15]:
# plot the combined corrected data
print("Plotting combined corrected data")
plot_res <- plot_diagnostic(corrected_expr, all_metadata, "Combined Corrected",
                            log_transform = TRUE, with_rowname = TRUE)
layout <- (plot_res[[1]] + plot_res[[2]] ) / 
          (plot_res[[3]] )
ggsave("after/diagnostic_plot_corrected.png", 
            plot = layout, width = 12, height = 12)


[1] "Plotting combined corrected data"
[1] "..plotting.."


Warning message:
“`aes_string()` was deprecated in ggplot2 3.0.0.
ℹ Please use tidy evaluation idioms with `aes()`.
ℹ See also `vignette("ggplot2-in-packages")` for more information.”
No id variables; using all as measure variables



In [16]:
# write out the corrected expression data
write.table(corrected_expr %>% rownames_to_column("gene_id"), "after/all_corrected_R_expr.tsv", sep = "\t", 
            quote = FALSE, col.names = TRUE, row.names = FALSE)